In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import numpy as np
import pandas as pd
from transformers import BertTokenizer
import nltk
import string
import re
import sys
from bs4 import BeautifulSoup
import requests
import time
import spacy
from collections import Counter

In [2]:
options = Options()
options.add_argument('headless');
driver = webdriver.Chrome(executable_path=r'D:\Downloads\chromedriver_win32\chromedriver.exe', options = options)

In [3]:
skills = ['Deep Learning', 'Convolutional Neural Networks', 'TensorFlow', 'Machine Learning','Algorithms','Computer Vision','Deep Reinforcement Learning','Computer Science','Python','Matlab','Recurrent Neural Networks','Django','Java','Keras','PyTorch','Neural Networks','Edge Computing','OpenCV','Scikit-Learn','Natural Language Processing','Object Detection','Reinforcement Learning','Generative Adverserial Networks']
skills_df = pd.read_csv("../data/skills.csv")
skills_df = skills_df.drop(columns = ["Unnamed: 0"])
skills_df.head(25)

,skills,summary
0,Deep Learning,Deep learning (also known as deep structured ...
1,Convolutional Neural Networks,"In deep learning, a convolutional neural netwo..."
2,TensorFlow,TensorFlow is a free and open-source software ...
3,Machine Learning,Machine learning (ML) is the study of computer...
4,Algorithms,"In mathematics and computer science, an algori..."
5,Computer Vision,Computer vision is an interdisciplinary scient...
6,Deep Reinforcement Learning,Deep reinforcement learning (deep RL) is a sub...
7,Computer Science,"Computer science is the study of computation, ..."
8,Python,NaN
9,Matlab,"MATLAB (an abbreviation of ""MATrix LABoratory""..."


#### Using Bert Tokenizer

In [4]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
def bertTokens(text):
    bertTokens = bert_tokenizer.tokenize(text)
    return bertTokens

100%|██████████████████████████████████████████████████████████████████████| 231508/231508 [00:00<00:00, 1284584.60B/s]


In [5]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

def remove_punctuation(text):
    try:
        punctuationfree = "".join([i for i in text if i not in string.punctuation])
    except:
        return " "
    return punctuationfree

def lower_case(text):
    try:
        lower = text.lower()
    except:
        print("Lower Case failed?")
        return ""
    return lower

def tokenization(text):
    tokens = nltk.word_tokenize(text)
    return tokens

In [28]:
bert_vocab = list(bert_tokenizer.vocab.keys())
def rare_word_extractor(word_list):
    rare_words = {}
    for word in word_list:
        if word in bert_vocab:
            ind = bert_vocab.index(word)
            if ind > 2700 and lemmatization([word])[0] != "-PRON-":
                rare_words[word] = ind
    rare_words_list = list(rare_words.keys())
    return rare_words_list
skills_df['clean_summary'] = skills_df['summary'].apply(lambda x:remove_punctuation(x)) 
skills_df['clean_summary'] = skills_df['clean_summary'].apply(lambda x:lower_case(x)) 
skills_df['tokens'] = skills_df['clean_summary'].apply(lambda x:tokenization(x)) 
skills_df['lemma_tokens'] = skills_df['tokens'].apply(lambda x:lemmatization(x))
skills_df['rare_words'] = skills_df['lemma_tokens'].apply(lambda x:rare_word_extractor(x)) 
skills_df.head()

,skills,summary,clean_summary,tokens,lemma_tokens,rare_words
0,Deep Learning,Deep learning (also known as deep structured ...,deep learning also known as deep structured l...,"[deep, learning, also, known, as, deep, struct...","[deep, learn, also, know, as, deep, structure,...","[deep, learn, structure, broad, machine, metho..."
1,Convolutional Neural Networks,"In deep learning, a convolutional neural netwo...",in deep learning a convolutional neural networ...,"[in, deep, learning, a, convolutional, neural,...","[in, deep, learn, a, convolutional, neural, ne...","[deep, learn, neural, network, cnn, artificial..."
2,TensorFlow,TensorFlow is a free and open-source software ...,tensorflow is a free and opensource software l...,"[tensorflow, is, a, free, and, opensource, sof...","[tensorflow, be, a, free, and, opensource, sof...","[software, library, machine, learn, artificial..."
3,Machine Learning,Machine learning (ML) is the study of computer...,machine learning ml is the study of computer a...,"[machine, learning, ml, is, the, study, of, co...","[machine, learn, ml, be, the, study, of, compu...","[machine, learn, ml, study, computer, algorith..."
4,Algorithms,"In mathematics and computer science, an algori...",in mathematics and computer science an algorit...,"[in, mathematics, and, computer, science, an, ...","[in, mathematic, and, computer, science, an, a...","[computer, algorithm, listen, finite, sequence..."


In [29]:
for index, row in skills_df.iterrows():
    print(row['rare_words'],"\n\n\n")

['deep', 'learn', 'structure', 'broad', 'machine', 'method', 'base', 'artificial', 'neural', 'network', 'representation', 'supervise', 'architecture', 'belief', 'reinforcement', 'apply', 'computer', 'vision', 'speech', 'recognition', 'natural', 'processing', 'translation', 'drug', 'medical', 'image', 'analysis', 'material', 'inspection', 'produce', 'result', 'comparable', 'expert', 'inspire', 'distribute', 'communication', 'node', 'biological', 'difference', 'brain', 'specifically', 'tend', 'static', 'symbolic', 'organism', 'dynamic', 'plastic', 'adjective', 'refer', 'multiple', 'layer', 'linear', 'universal', 'activation', 'function', 'hide', 'width', 'modern', 'variation', 'concerned', 'bound', 'size', 'permits', 'practical', 'application', 'implementation', 'retain', 'theoretical', 'mild', 'condition', 'permit', 'widely', 'inform', 'model', 'sake', 'efficiency'] 



['deep', 'learn', 'neural', 'network', 'cnn', 'artificial', 'commonly', 'apply', 'analyze', 'visual', 'imagery', 'shif

In [30]:
history = pd.read_csv("../data/web_history/synthetic_490.csv")
history = history.drop(columns = ["Unnamed: 0"])
history

,URL,TITLE,Visited_time
0,https://www.tensorflow.org/tutorials/images/cnn,Convolutional Neural Network (CNN) | TensorF...,2021-12-03 03:39:00
1,https://www.tensorflow.org/,TensorFlow,2021-12-03 03:39:00
2,https://en.wikipedia.org/wiki/TensorFlow,TensorFlow - Wikipedia,2021-12-03 03:38:00
3,https://github.com/tensorflow/tensorflow,tensorflow/tensorflow: An Open Source Machine ...,2021-12-03 03:38:00
4,https://www.tensorflow.org/,TensorFlow,2021-12-03 03:38:00
...,...,...,...
486,https://en.wikipedia.org/w/index.php?search=Co...,Convolutional neural network - Wikipedia,2021-12-03 02:11:00
487,https://en.wikipedia.org/wiki/Microsoft_Windows,Microsoft Windows - Wikipedia,2021-12-03 02:11:00
488,https://en.wikipedia.org/wiki/Command-line_int...,Command-line interface - Wikipedia,2021-12-03 02:10:00
489,https://en.wikipedia.org/wiki/Graphical_user_i...,Graphical user interface - Wikipedia,2021-12-03 02:09:00


In [31]:
work = []
for i in  range(0,491):
    work.append("WORK")
history['LABEL'] = pd.Series(work)
history.head()

,URL,TITLE,Visited_time,LABEL
0,https://www.tensorflow.org/tutorials/images/cnn,Convolutional Neural Network (CNN) | TensorF...,2021-12-03 03:39:00,WORK
1,https://www.tensorflow.org/,TensorFlow,2021-12-03 03:39:00,WORK
2,https://en.wikipedia.org/wiki/TensorFlow,TensorFlow - Wikipedia,2021-12-03 03:38:00,WORK
3,https://github.com/tensorflow/tensorflow,tensorflow/tensorflow: An Open Source Machine ...,2021-12-03 03:38:00,WORK
4,https://www.tensorflow.org/,TensorFlow,2021-12-03 03:38:00,WORK


In [32]:
history['H1'] = np.empty((len(history), 0)).tolist()
history['H2'] = np.empty((len(history), 0)).tolist()
history['H3'] = np.empty((len(history), 0)).tolist()
history['H4'] = np.empty((len(history), 0)).tolist()
history['H5'] = np.empty((len(history), 0)).tolist()
history['H6'] = np.empty((len(history), 0)).tolist()
history['SPAN'] = np.empty((len(history), 0)).tolist()
history['p'] = np.empty((len(history), 0)).tolist()
history.head()

,URL,TITLE,Visited_time,LABEL,H1,H2,H3,H4,H5,H6,SPAN,p
0,https://www.tensorflow.org/tutorials/images/cnn,Convolutional Neural Network (CNN) | TensorF...,2021-12-03 03:39:00,WORK,[],[],[],[],[],[],[],[]
1,https://www.tensorflow.org/,TensorFlow,2021-12-03 03:39:00,WORK,[],[],[],[],[],[],[],[]
2,https://en.wikipedia.org/wiki/TensorFlow,TensorFlow - Wikipedia,2021-12-03 03:38:00,WORK,[],[],[],[],[],[],[],[]
3,https://github.com/tensorflow/tensorflow,tensorflow/tensorflow: An Open Source Machine ...,2021-12-03 03:38:00,WORK,[],[],[],[],[],[],[],[]
4,https://www.tensorflow.org/,TensorFlow,2021-12-03 03:38:00,WORK,[],[],[],[],[],[],[],[]


In [33]:
def process_url(url):
    driver.get(url)
    h1 = driver.find_elements_by_tag_name("h1")
    h2 = driver.find_elements_by_tag_name("h2")
    h3 = driver.find_elements_by_tag_name("h3")
    h4 = driver.find_elements_by_tag_name("h4")
    h5 = driver.find_elements_by_tag_name("h5")
    h6 = driver.find_elements_by_tag_name("h6")
    span = driver.find_elements_by_tag_name("span")
    p = driver.find_elements_by_tag_name("p")
    return (h1,h2,h3,h4,h5,h6,span,p)

def extract_rare_words_from_sel_objects(tag):
    rare_words = []
    if tag != None:
        for text in tag:
            rare_words.extend(rare_word_extractor(lemmatization(tokenization(lower_case(remove_punctuation(text.text))))))
    return rare_words

In [34]:
for index,row in history.iterrows():
    print("URL INDEX: ", index, "  LABEL: ", row['LABEL'] )
    if index >= 10:
        break
    
    if row['LABEL'] != "MISC":
        try:
            (h1,h2,h3,h4,h5,h6,span,p) = process_url(row['URL'])
        except:
            print("URL: ", row['URL'], "\n\n")
            print("Index:", index,"Exception while processing url")
            row['H1'] = []
            row['H2'] = []
            row['H3'] = []
            row['H4'] = []
            row['H5'] = []
            row['H6'] = []
            row['SPAN'] = []
            row['p'] = []
            continue
        time.sleep(1)

        try:
            rare_words_h1 = extract_rare_words_from_sel_objects(h1)
        except:
            print("Index:", index,"Exception while extracting h1")
            rare_words_h1 = []

        try:
            rare_words_h2 = extract_rare_words_from_sel_objects(h2)
        except:
            print("Index:", index,"Exception while extracting h2")
            rare_words_h2 = []

        try:
            rare_words_h3 = extract_rare_words_from_sel_objects(h3)
        except:
            print("Index:", index,"Exception while extracting h3")
            rare_words_h3 = []
        
        try:
            rare_words_h4 = extract_rare_words_from_sel_objects(h4)
        except:
            print("Index:", index,"Exception while extracting h4")
            rare_words_h4 = []
        
        try:
            rare_words_h5 = extract_rare_words_from_sel_objects(h5)
        except:
            print("Index:", index,"Exception while extracting h5")
            rare_words_h5 = []
        
        try:
            rare_words_h6 = extract_rare_words_from_sel_objects(h6)
        except:
            print("Index:", index,"Exception while extracting h6")
            rare_words_h6 = []
        
        try:
            rare_words_span = extract_rare_words_from_sel_objects(span)
        except:
            print("Index:", index,"Exception while extracting span")
            rare_words_span = []
        

        try:
            rare_words_p  = extract_rare_words_from_sel_objects(p)
        except:      
            print("Index:", index,"Exception while extracting p")
            rare_words_p = []
        
        print("URL: ", row['URL'], "\n\n")
        print("H1:  ",rare_words_h1, str("\n"))
        print("H2:  ",rare_words_h2,str("\n"))
        print("H3:  ",rare_words_h3,str("\n"))
        print("H4:  ",rare_words_h4, str("\n"))
        print("H5:  ",rare_words_h5,str("\n"))
        print("H6:  ",rare_words_h6,str("\n"))
        print("SPAN:  ",rare_words_span,str("\n"))
        print("P:  ",rare_words_p,str("\n"))
    else:
        rare_words_h1 = []
        rare_words_h2 = []
        rare_words_h3 = []
        rare_words_h4 = []
        rare_words_h5 = []
        rare_words_h6 = []
        rare_words_span = []
        rare_words_p = []    
    
    row['H1'] = rare_words_h1
    row['H2'] = rare_words_h2
    row['H3'] = rare_words_h3
    row['H4'] = rare_words_h4
    row['H5'] = rare_words_h5
    row['H6'] = rare_words_h6
    row['SPAN'] = rare_words_span
    row['p'] = np.array(rare_words_p)

URL INDEX:  0   LABEL:  WORK
URL:  https://www.tensorflow.org/tutorials/images/cnn 


H1:   ['neural', 'network', 'cnn'] 

H2:   [] 

H3:   ['import', 'download', 'prepare', 'verify', 'create', 'base', 'add', 'dense', 'layer', 'train', 'model', 'evaluate', 'model', 'stay', 'connect'] 

H4:   [] 

H5:   [] 

H6:   [] 

SPAN:   ['page', 'page', 'import', 'download', 'prepare', 'verify', 'create', 'base', 'add', 'dense', 'layer', 'train', 'model', 'evaluate', 'model', 'import', 'import', 'layer', 'model', 'import', 'pixel', 'value', 'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck', 'figure', 'range', 'grid', 'false', 'label', 'happen', 'arrays', 'extra', 'index', 'model', 'model', 'sequential', 'model', 'add', 'layer', '32', 'activation', '32', '32', 'model', 'add', 'layer', 'model', 'add', 'layer', '64', 'activation', 'model', 'add', 'layer', 'model', 'add', 'layer', '64', 'activation', 'model', 'summary', 'model', 'add', 'layer', 'model', 'add', '

URL:  https://github.com/tensorflow/tensorflow 


H1:   [] 

H2:   ['commit', 'stat', 'file', 'install', 'contribution', 'continuous', 'build', 'status', 'resource', 'license', 'release', '152', 'package', 'contributor'] 

H3:   ['official', 'builds', 'topic', 'resource', 'license', 'code', 'conduct', 'citation'] 

H4:   ['try'] 

H5:   [] 

H6:   [] 

SPAN:   ['star', 'code', 'issue', 'pull', 'request', '168', 'action', 'master', 'code', 'update', 'generate', 'function', 'ops', 'ci', 'python', '39', 'action', 'update', 'generate', 'function', 'ops', 'update', 'dependency', 'revision', 'tool', 'merge', 'pull', 'request', 'add', 'option', 'upgrade', '421', 'ignore', 'build', 'file', 'generate', 'script', 'add', 'root', 'add', 'clean', 'doi', 'author', 'format', 'author', 'file', 'build', 'nfc', 'internal', 'polish', 'file', 'add', 'replace', 'update', 'fix', 'break', 'sync', 'merge', 'pull', 'request', 'internal', 'merge', 'pull', 'request', 'license', 'remove', 'duplicate', 'license', 

URL:  https://www.tensorflow.org/tutorials/images/cnn 


H1:   ['neural', 'network', 'cnn'] 

H2:   [] 

H3:   ['import', 'download', 'prepare', 'verify', 'create', 'base', 'add', 'dense', 'layer', 'train', 'model', 'evaluate', 'model', 'stay', 'connect'] 

H4:   [] 

H5:   [] 

H6:   [] 

SPAN:   ['page', 'page', 'import', 'download', 'prepare', 'verify', 'create', 'base', 'add', 'dense', 'layer', 'train', 'model', 'evaluate', 'model', 'import', 'import', 'layer', 'model', 'import', 'pixel', 'value', 'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck', 'figure', 'range', 'grid', 'false', 'label', 'happen', 'arrays', 'extra', 'index', 'model', 'model', 'sequential', 'model', 'add', 'layer', '32', 'activation', '32', '32', 'model', 'add', 'layer', 'model', 'add', 'layer', '64', 'activation', 'model', 'add', 'layer', 'model', 'add', 'layer', '64', 'activation', 'model', 'summary', 'model', 'add', 'layer', 'model', 'add', 'layer', 'dense', '64', 'activ

URL:  https://www.ibm.com/cloud/learn/convolutional-neural-networks 


H1:   ['neural', 'network', 'neural', 'network'] 

H2:   ['neural', 'network', 'neural', 'network', 'type', 'neural', 'network', 'neural', 'network', 'computer', 'vision', 'neural', 'network', 'ibm'] 

H3:   ['layer', 'pool', 'layer', 'layer'] 

H4:   [] 

H5:   [] 

H6:   [] 

SPAN:   ['contact', 'neural', 'network', 'ibm', 'cloud', 'ibm', 'cloud', 'neural', 'network', 'learn', 'neural', 'network', 'image', 'classification', 'object', 'recognition', 'task'] 

P:   ['neural', 'network', 'learn', 'hub', 'article', 'subset', 'machine', 'deep', 'algorithms', 'comprise', 'node', 'layer', 'contain', 'input', 'hide', 'output', 'connect', 'associated', 'weight', 'threshold', 'individual', 'specify', 'value', 'activate', 'send', 'otherwise', 'pass', 'primarily', 'focus', 'network', 'article', 'type', 'neural', 'net', 'example', 'commonly', 'natural', 'processing', 'speech', 'recognition', 'whereas', 'utilize', 'classificati

In [13]:
for index, row in history.iterrows():
    if index>=10:
        break
    print("URL INDEX: ", index, "ROW: ",np.array(list(row['H1'])))

URL INDEX:  0 ROW:  ['convolutional' 'neural' 'network' 'cnn']
URL INDEX:  1 ROW:  []
URL INDEX:  2 ROW:  ['tensorflow']
URL INDEX:  3 ROW:  ['tensorflow']
URL INDEX:  4 ROW:  []
URL INDEX:  5 ROW:  ['accessibility' 'link' 'search' 'mode' 'search' 'result' 'page'
 'navigation' 'footer' 'link']
URL INDEX:  6 ROW:  ['accessibility' 'link' 'search' 'mode' 'search' 'result' 'page'
 'navigation' 'footer' 'link']
URL INDEX:  7 ROW:  ['convolutional' 'neural' 'network' 'cnn']
URL INDEX:  8 ROW:  ['convolutional' 'layer' 'deep' 'learn' 'neural' 'network']
URL INDEX:  9 ROW:  ['convolutional' 'neural' 'network' 'convolutional' 'neural' 'network']


In [14]:
skill_rare_words = []
for i, skill in skills_df.iterrows():
    skill_rare_words.extend(skill['rare_words'])
print(skill_rare_words)

['deep', 'learn', 'structure', 'broad', 'machine', 'method', 'base', 'artificial', 'neural', 'network', 'representation', 'supervise', 'semisupervised', 'unsuperviseddeeplearne', 'architecture', 'belief', 'reinforcement', 'recurrent', 'convolutional', 'apply', 'computer', 'vision', 'speech', 'recognition', 'natural', 'processing', 'translation', 'bioinformatics', 'drug', 'medical', 'image', 'analysis', 'material', 'inspection', '-PRON-', 'produce', 'result', 'comparable', 'surpass', 'expert', 'performanceartificial', 'anns', 'inspire', 'distribute', 'communication', 'node', 'biological', 'difference', 'brain', 'specifically', 'tend', 'static', 'symbolic', 'organism', 'dynamic', 'plastic', 'analoguethe', 'adjective', 'refer', 'multiple', 'layer', 'linear', 'perceptron', 'universal', 'classifier', 'nonpolynomial', 'activation', 'function', 'hide', 'unbounded', 'width', 'modern', 'variation', 'concerned', 'bound', 'size', 'permits', 'practical', 'application', 'optimize', 'implementation'

In [35]:
def word_match_count(skill_list, fetched_list):
    count = 0
    for word in fetched_list:
        if word in skill_list:
            count = count + fetched_list[word]
        if debug:
            print(word)
    return count

def percent_word_count_match(skill_list, fetched_list):
    count = word_match_count(skill_list,fetched_list)
    if count > 0 and len(fetched_list) > 0:
        return (count/len(fetched_list)) * 100
    else:
        return 0

In [36]:
debug = False
for index, row in history.iterrows():
    if index>=10:
        break
    print("\nURL INDEX: ", index, "  Label: ", row['LABEL'] )
    h1 = Counter(row['H1'])
    h2 = Counter(row['H2'])
    h3 = Counter(row['H3'])
    h4 = Counter(row['H4'])
    h5 = Counter(row['H5'])
    h6 = Counter(row['H6'])
    span = Counter(row['SPAN'])
    p = Counter(row['p'])
    print("\nH1 :", percent_word_count_match(skill_rare_words, h1))
    print("H2 :", percent_word_count_match(skill_rare_words, h2))
    print("H3 :", percent_word_count_match(skill_rare_words, h3))
    print("H4 :", percent_word_count_match(skill_rare_words, h4))
    print("H5 :", percent_word_count_match(skill_rare_words, h5))
    print("H6 :", percent_word_count_match(skill_rare_words, h6))
    print("SPAN :", percent_word_count_match(skill_rare_words, span))
    print("P :", percent_word_count_match(skill_rare_words, p))
    


URL INDEX:  0   Label:  WORK

H1 : 100.0
H2 : 0
H3 : 57.14285714285714
H4 : 0
H5 : 0
H6 : 0
SPAN : 101.88679245283019
P : 73.19587628865979

URL INDEX:  1   Label:  WORK

H1 : 0
H2 : 50.0
H3 : 20.0
H4 : 62.5
H5 : 0
H6 : 0
SPAN : 30.0
P : 63.2

URL INDEX:  2   Label:  WORK

H1 : 0
H2 : 33.33333333333333
H3 : 63.63636363636363
H4 : 100.0
H5 : 0
H6 : 0
SPAN : 68.9922480620155
P : 93.52112676056338

URL INDEX:  3   Label:  WORK

H1 : 0
H2 : 28.57142857142857
H3 : 25.0
H4 : 0
H5 : 0
H6 : 0
SPAN : 54.09836065573771
P : 54.166666666666664

URL INDEX:  4   Label:  WORK

H1 : 0
H2 : 50.0
H3 : 20.0
H4 : 62.5
H5 : 0
H6 : 0
SPAN : 30.0
P : 63.2

URL INDEX:  5   Label:  WORK

H1 : 25.0
H2 : 160.0
H3 : 50.0
H4 : 0
H5 : 0
H6 : 0
SPAN : 87.83783783783784
P : 0

URL INDEX:  6   Label:  WORK

H1 : 25.0
H2 : 160.0
H3 : 50.0
H4 : 0
H5 : 0
H6 : 0
SPAN : 87.83783783783784
P : 0

URL INDEX:  7   Label:  WORK

H1 : 100.0
H2 : 0
H3 : 57.14285714285714
H4 : 0
H5 : 0
H6 : 0
SPAN : 101.88679245283019
P : 73.1958

In [37]:
def test_url(url):
    try:
        (h1,h2,h3,h4,h5,h6,span,p) = process_url(url)
        rare_words_h1 = extract_rare_words_from_sel_objects(h1)
        rare_words_h2 = extract_rare_words_from_sel_objects(h2)
        rare_words_h3 = extract_rare_words_from_sel_objects(h3)
        rare_words_h4 = extract_rare_words_from_sel_objects(h4)
        rare_words_h5 = extract_rare_words_from_sel_objects(h5)
        rare_words_h6 = extract_rare_words_from_sel_objects(h6)
        rare_words_span = extract_rare_words_from_sel_objects(span)
        rare_words_p = extract_rare_words_from_sel_objects(p)
        h1 = Counter(rare_words_h1)
        h2 = Counter(rare_words_h2)
        h3 = Counter(rare_words_h3)
        h4 = Counter(rare_words_h4)
        h5 = Counter(rare_words_h5)
        h6 = Counter(rare_words_h6)
        span = Counter(rare_words_span)
        p = Counter(rare_words_p)
        print("\nH1 :", percent_word_count_match(skill_rare_words, h1))
        print("H2 :", percent_word_count_match(skill_rare_words, h2))
        print("H3 :", percent_word_count_match(skill_rare_words, h3))
        print("H4 :", percent_word_count_match(skill_rare_words, h4))
        print("H5 :", percent_word_count_match(skill_rare_words, h5))
        print("H6 :", percent_word_count_match(skill_rare_words, h6))
        print("SPAN :", percent_word_count_match(skill_rare_words, span))
        print("P :", percent_word_count_match(skill_rare_words, p))
    except:
        print("Thrown Exception at testing the url")

In [38]:
debug = False
print("\nURL 1 Score: ")
# False
test_url("https://www.gov.uk/uk-border-control") # travel website
print("\nURL 2 Score: ")
# True
test_url("https://machinelearningmastery.com/what-are-generative-adversarial-networks-gans/") # About GANs, which is not one of the scraped skills
print("\nURL 3 Score: ")
# True
test_url("https://builtin.com/robotics") # About Robotics, not listed skill but related
print("\nURL 4 Score: ")
# False - Stock market not related to my skills 
test_url("https://www.financialexpress.com/market/icici-bank-hdfc-tata-motors-among-top-stock-ideas-from-jefferies-expect-around-30-returns/2378875/")


URL 1 Score: 

H1 : 0
H2 : 38.46153846153847
H3 : 0
H4 : 0
H5 : 0
H6 : 0
SPAN : 30.0
P : 28.57142857142857

URL 2 Score: 

H1 : 50.0
H2 : 83.33333333333334
H3 : 85.71428571428571
H4 : 77.77777777777779
H5 : 0
H6 : 0
SPAN : 51.66666666666667
P : 169.73415132924333

URL 3 Score: 

H1 : 100.0
H2 : 77.27272727272727
H3 : 68.42105263157895
H4 : 0
H5 : 0
H6 : 0
SPAN : 44.126074498567334
P : 59.80861244019139

URL 4 Score: 

H1 : 28.57142857142857
H2 : 31.57894736842105
H3 : 0
H4 : 0
H5 : 0
H6 : 0
SPAN : 28.888888888888886
P : 24.342105263157894


In [40]:
import time
start = time.time()
test_url("https://www.gov.uk/uk-border-control") # travel website
end = time.time()
print(end-start)


H1 : 0
H2 : 38.46153846153847
H3 : 0
H4 : 0
H5 : 0
H6 : 0
SPAN : 30.0
P : 28.57142857142857
1.2374536991119385


In [ ]:
other_url_df = pd.read_excel("../data/web_history/other_website_history_xl.xls")
other_url_df.head()
other_urls = list(other_url_df["URL"])

In [ ]:
for i, url in enumerate(other_urls):
    print("Index: ",i)
    test_url(url)
    if i >= 100:
        break